In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()

In [ ]:
### deaths ####

dead_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'
dead_json = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_dead = requests.post(dead_url,json=dead_json)
r_dead.status_code

In [ ]:
json_dead = r_dead.json()
json_dead['data'][0]

In [ ]:
### population ###
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'
data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "1968",
          "1969",
          "1970",
          "1971",
          "1972",
          "1973",
          "1974",
          "1975",
          "1976",
          "1977",
          "1978",
          "1979",
          "1980",
            "1981",
            "1982",
            "1983",
            "1984",
            "1985",
            "1986",
            "1987",
            "1988",
            "1989",
            "1990",
            "1991",
            "1992",
            "1993",
            "1994",
            "1995",
            "1996",
            "1997",
            "1998",
            "1999",
            "2000",
            "2001",
            "2002",
            "2003",
            "2004",
            "2005",
            "2006",
            "2007",
            "2008",
            "2009",
            "2010",
            "2011",
            "2012",
            "2013",
          "2014",  
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

r = requests.post(url,json=data)
print (r.status_code)
json_pop = r.json()

json_pop['data'][0]

In [ ]:
def parse_json(json):
    
    nr_keys = len(json['data'][0]['key'])
    nr_vals = len(json['data'][0]['values'])
    
    
    items = len(json['data'])
    data_list = []

    for i in range(items):
        val_list = []
        
        for j,k in enumerate(range(nr_keys)):
            val_list.append(json['data'][i]['key'][j])
            
        for j,v in enumerate(range(nr_vals)):
            val_list.append(json['data'][i]['values'][j])
            
        data_list.append(val_list)

    return data_list
    
dead_df = pd.DataFrame(parse_json(json_dead))
dead_df.drop(0,axis=1,inplace=True)
dead_df    

In [ ]:
dead_df.columns = ['age','gender','year','dead']
dead_df['age'] = pd.Categorical(dead_df['age'],['-4','5-9','10-14','15-19','20-24',
                                                 '25-29','30-34','35-39','40-44',
                                                 '45-49','50-54','55-59','60-64','65-69',
                                                 '70-74','75-79','80-84','85-89',
                                                 '90-94','95-99','100+'])

dead_df['dead'] = dead_df['dead'].astype(int)
dead_df

In [ ]:
dead_df_men_women = dead_df.groupby(['year','age']).sum()

dead_df_men_women = dead_df_men_women.unstack()
dead_df_men_women = dead_df_men_women.droplevel(0,axis=1)
dead_df_men_women.info()

In [ ]:
dead_df_men_women

In [ ]:
pop_df = pd.DataFrame(parse_json(json_pop))
pop_df.columns = ['age','gender','year','pop']
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df['age'] = pd.Categorical(pop_df['age'],['-4','5-9','10-14','15-19','20-24',
                                                 '25-29','30-34','35-39','40-44',
                                                 '45-49','50-54','55-59','60-64','65-69',
                                                 '70-74','75-79','80-84','85-89',
                                                 '90-94','95-99','100+'])

pop_df

In [ ]:
pop_df = pop_df.groupby(['year','age']).sum()
pop_df

In [ ]:
pop_df_men_women = pop_df.unstack().droplevel(0,axis=1)
pop_df_men_women

In [ ]:

### NOT REAL MORTALITY SINCE WE ARE ADDING MORTALITIES FOR DIFFERENT AGE GROUPS ! ###

age_grp_mortality = dead_df_men_women / pop_df_men_women
age_grp_mortality

In [ ]:
avg_mortality_per_age_grp = age_grp_mortality.mean()
avg_mortality_per_age_grp

In [ ]:
avg_mortality_per_age_grp.plot(figsize=(18,12),style='o--')

In [ ]:
std_pop = pop_df_men_women.loc['2019']

In [ ]:
strata_abs_deaths = age_grp_mortality * std_pop
strata_abs_deaths

In [ ]:
strata_abs_deaths

In [ ]:
strata_abs_deaths.plot(kind='bar',stacked=True,figsize=(18,12))

In [ ]:
tot_strat_deaths = strata_abs_deaths.sum(axis=1)
tot_strat_deaths

In [ ]:
age_adj_mortality = tot_strat_deaths / std_pop.sum()
age_adj_mortality

In [ ]:
age_adj_mortality.plot(kind='bar',figsize=(18,12))
_= plt.yticks(np.arange(0,0.019,0.001))